In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming your data is organized in two folders: train and validation
train_data_dir = 'train'
validation_data_dir = 'val'
img_width, img_height = 150, 150  # Adjust dimensions as needed
batch_size = 18
epochs = 25

# Choose a CNN architecture
# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


# ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False
num_classes = train_generator.num_classes 
print(num_classes)

# Add classification layers on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])



# Train the model
hist=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)
model.save("resnet1.keras")

loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Found 27437 images belonging to 7 classes.
Found 2238 images belonging to 7 classes.
7
Epoch 1/25
  33/1524 [..............................] - ETA: 7:42 - loss: 4.6422 - accuracy: 0.3350

C:\Users\Apkr\anaconda3\envs\tf\lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1524/1524 [==============================] - 560s 366ms/step - loss: 1.6953 - accuracy: 0.4509 - val_loss: 1.8506 - val_accuracy: 0.2056
Epoch 2/25
1524/1524 [==============================] - 604s 396ms/step - loss: 1.6145 - accuracy: 0.4591 - val_loss: 1.9750 - val_accuracy: 0.2164
Epoch 3/25
1524/1524 [==============================] - 614s 403ms/step - loss: 1.6093 - accuracy: 0.4618 - val_loss: 1.9655 - val_accuracy: 0.2249
Epoch 4/25
1524/1524 [==============================] - 1250s 820ms/step - loss: 1.5974 - accuracy: 0.4652 - val_loss: 1.9262 - val_accuracy: 0.2267
Epoch 5/25
1524/1524 [==============================] - 1444s 947ms/step - loss: 1.5862 - accuracy: 0.4703 - val_loss: 1.7926 - val_accuracy: 0.3087
Epoch 6/25
1524/1524 [==============================] - 782s 513ms/step - loss: 1.5735 - accuracy: 0.4790 - val_loss: 1.8290 - val_accuracy: 0.2572
Epoch 7/25
1524/1524 [==============================] - 719s 472ms/step - loss: 1.5678 - accuracy: 0.4804 - val_loss: 1.8

In [ ]:
import numpy as np
import tkinter as tk
from tkinter import filedialog, Label, Button, Frame, ttk
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

class DeepLearningModel:
    def __init__(self, model_path):
        self.model = load_model(model_path)
    
    def load_and_preprocess_image(self, img_path, target_size=(150, 150)):
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array
    
    def predict(self, img_path):
        img_array = self.load_and_preprocess_image(img_path)
        predictions = self.model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)
        return predicted_class[0]

class ImageClassifierUI:
    def __init__(self, root, model):
        self.root = root
        self.model = model
        self.root.title("Image Classification with Keras")
        self.root.geometry("500x400")

        # Create a main frame
        self.main_frame = Frame(root, bg="#f0f0f0")
        self.main_frame.pack(fill="both", expand=True, padx=20, pady=20)

        # Title
        self.title_label = Label(self.main_frame, text="Image Classification", font=("Helvetica", 18, "bold"), bg="#f0f0f0")
        self.title_label.pack(pady=10)

        # Image display area
        self.img_frame = Frame(self.main_frame, bg="#ffffff", borderwidth=2, relief="sunken")
        self.img_frame.pack(pady=10)
        self.img_label = Label(self.img_frame)
        self.img_label.pack()

        # Load button
        self.load_button = Button(self.main_frame, text="Load Image", command=self.load_image, bg="#4CAF50", fg="white", font=("Helvetica", 12))
        self.load_button.pack(pady=10)

        # Result label
        self.result_label = Label(self.main_frame, text="", font=("Helvetica", 14), bg="#f0f0f0")
        self.result_label.pack(pady=10)

        # Progress bar
        self.progress = ttk.Progressbar(self.main_frame, mode='indeterminate')
        self.progress.pack(pady=10)
        self.progress_text = Label(self.main_frame, text="", font=("Helvetica", 12), bg="#f0f0f0")
        self.progress_text.pack(pady=5)

        # To keep reference to image
        self.img_ref = None

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            self.show_image(file_path)
            self.predict_image(file_path)
    
    def show_image(self, file_path):
        img = Image.open(file_path)
        img.thumbnail((300, 300))
        img = ImageTk.PhotoImage(img)

        # Update image label
        self.img_label.config(image=img)
        self.img_label.image = img

        # Keep reference to the image object
        self.img_ref = img

    def predict_image(self, file_path):
        self.progress.start()
        self.progress_text.config(text="Processing...")
        self.root.update_idletasks()
        
        predicted_class = self.model.predict(file_path)

        # Simulate some processing delay
        self.root.after(2000, lambda: self.update_result(predicted_class))

    def update_result(self, predicted_class):
        self.progress.stop()
        self.progress_text.config(text="Done!")
        self.result_label.config(text=f'Predicted class: {predicted_class}')

if __name__ == "__main__":
    # Initialize the deep learning model with the path to your model
    deep_learning_model = DeepLearningModel('resnet_model.h5')

    # Initialize the Tkinter root window
    root = tk.Tk()

    # Initialize the UI with the root window and the deep learning model
    app = ImageClassifierUI(root, deep_learning_model)

    # Start the Tkinter event loop
    root.mainloop()
